Data Collection using webscraping and API
Name: Samujjal Seal Sarma
    

# Scraping Youtube - Matching Videos Details
  Steps:
     (Step 1) : Set Webdriver path for chromedriver accordingly
     (Step 2) : Set Control_key value accordingly. If running on MAC, set Control_key = Keys.COMMAND. If running on
               windows, set Control_key=Keys.COMMAND
     (Step 3) : Using chromedriver, open youtube url
     (Step 4) : Enter search_text as 'Kishore Kumar' and submit page
     (Step 5) : In search results, run a loop 10 times to fetch each video search result details 
     (Step 6) : Open each search result url above, and open video in new window
     (Step 7) : Scroll down the video link to load comments. Check number of comments and load max 50 comments
     (Step 8) : Read below list of values for each video into rowlist
             [href, title, subscriptions,viewsCount, lastUpdated, comments, author, postedTime, replyCount, overallvote]           
     (Step 9) : Create dataframe out of rowlist and columnlist variables above
     (Step 10): Write data from dataframe created above to output_scraping.csv file

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import time
import csv
import pandas as pd

In [4]:
url = "https://www.youtube.com"

# Set chrome driver path accordingly
driver = webdriver.Chrome('D://ISB CBA/Term 1/DC/chromedriver.exe')
# For running this code on MAC Machine, Set Control_key to below value. On MAC, command+enter is used to open new window url
#Control_key = Keys.COMMAND
#For running this code on Windows Machine, Set Control_key to below value. On Windows, control+enter is used to open new window url
Control_key = Keys.CONTROL

driver.get(url)
search_text = "Kishore kumar"

#driver.maximize_window()
textbox = driver.find_element_by_id("search").send_keys(search_text)
button = driver.find_element_by_id("search-icon-legacy")

button.submit()
print("Submitted search for search_text")
#page1_results = driver.find_elements(By.XPATH, '//*[@id="contents"]')
time.sleep(3)

videoCounter=1 
rowlist = []

while videoCounter <= 10:
    print("Video: ",videoCounter)
    search_list = []
    try:
        search_list = driver.find_elements_by_xpath('//*[@id="contents"]/ytd-video-renderer['+str(videoCounter)+']')[0]
        rowInitial = []
        try:#url
            rowInitial.append(search_list.find_element_by_xpath('.//*[@id="video-title"]').get_property("href"))
        except:
            rowInitial.append("")
        try:#subscription
            rowInitial.append(search_list.find_element_by_xpath('.//*[@id="byline"]/a').text)
        except:
            row.append("")
        try:#title
            rowInitial.append(search_list.find_element_by_xpath('.//*[@id="video-title"]').text)
        except:
            rowInitial.append("")
        try:#views count
            rowInitial.append(search_list.find_element_by_xpath('.//*[@id="metadata-line"]/span[1]').text)
        except:
            rowInitial.append("")
        try:#last updated
            rowInitial.append(search_list.find_element_by_xpath('.//*[@id="metadata-line"]/span[2]').text)
        except:
            rowInitial.append("")
        print("Opening Video: ",videoCounter)
        search_list.find_element_by_xpath('.//*[@id="video-title"]').send_keys(Control_key + Keys.RETURN)
        time.sleep(3)
        print("Switching driver to Video: ",videoCounter)
        driver.switch_to.window(driver.window_handles[1]);
        time.sleep(2)
        #scroll page till this first scroll element
        print("Switching page for Video: ",videoCounter)
        prev_scroll = driver.find_elements_by_xpath('//*[@id="more"]/yt-formatted-string')[0].location['y']
        driver.execute_script("window.scrollTo(0,"+ str(prev_scroll)+ ")")
        time.sleep(4)
        
        #get number of available comments
        print("Getting number of comments for Video: ",videoCounter)
        counts_element = driver.find_elements_by_xpath('//*[@id="count"]/yt-formatted-string')[0]
        no_of_comments = int(counts_element.text.split( )[0].replace(',',''))
        prev_scroll = counts_element.location['x']
        print("no_of_comments ", no_of_comments)
        if no_of_comments > 50:
            no_of_comments = 50
        
        #append comments of each link
        commentsCounter = 1
        print("no_of_comments ", no_of_comments)

        print('Loading comments')
        while commentsCounter <= no_of_comments: 
            row = []
            row = rowInitial.copy()
            try:
                next_comment = driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer['+str(commentsCounter)+']')[0]
                try:#comment
                    row.append(next_comment.find_elements_by_xpath('.//*[@id="content-text"]')[0].text)
                except:
                    row.append("")
                try:#author
                     row.append(next_comment.find_elements_by_xpath('.//*[@id="author-text"]')[0].text)
                except:
                    row.append("")            
                try:#published time
                    row.append(next_comment.find_elements_by_xpath('.//*[@id="published-time-text"]/a')[0].text)
                except:
                    row.append("")
                try:#reply count
                    row.append(next_comment.find_elements_by_xpath('.//*[@id="more"]/div/paper-button')[0].text)
                except:
                    row.append("")
                try:#vote count
                    row.append(next_comment.find_elements_by_xpath('.//*[@id="vote-count-middle"]')[0].text)
                except:
                    row.append("")
                rowlist.append(row)
                commentsCounter += 1
            except:
                #print("Next comments element not found, so loading again")
                i = 0
                while i < 2:
                    try:
                        parent_element = driver.find_elements_by_xpath('//*[@id="continuations"]')[1]
                        prev_scroll += parent_element.location['x']
                        driver.execute_script("window.scrollTo("+ str(prev_scroll) + "," + str(prev_scroll+parent_element.location['x']) + ")")                    
                        i+=1
                        time.sleep(2)
                    except:
                        if commentsCounter<=50:
                            try:
                                print("in except try ",driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer['+str(count1)+']')[0])
                                commentsCounter += 1
                            except:
                                commentsCounter = commentsCounter+1
               
    except Exception as e:
        print("Exception "+str(e))
    print('Finished comments extraction')   
    driver.close();
    driver.switch_to.window(driver.window_handles[0]);
    videoCounter += 1
#insert comments data into csv file
columns_list = ["href","title","subscriptions","viewsCount","lastUpdated","conmments","author","postedTime","replyCount","overallvote"]
data = pd.DataFrame(rowlist, columns=columns_list )   
data.to_csv('output_scraping.csv', index=False, columns=columns_list)
   

Submitted search for search_text
Video:  1
Opening Video:  1
Switching driver to Video:  1
Switching page for Video:  1
Getting number of comments for Video:  1
no_of_comments  1876
no_of_comments  50
Loading comments
Finished comments extraction
Video:  2
Opening Video:  2
Switching driver to Video:  2
Switching page for Video:  2
Getting number of comments for Video:  2
no_of_comments  2409
no_of_comments  50
Loading comments
Finished comments extraction
Video:  3
Opening Video:  3
Switching driver to Video:  3
Switching page for Video:  3
Getting number of comments for Video:  3
no_of_comments  648
no_of_comments  50
Loading comments
Finished comments extraction
Video:  4
Opening Video:  4
Switching driver to Video:  4
Switching page for Video:  4
Getting number of comments for Video:  4
no_of_comments  3245
no_of_comments  50
Loading comments
Finished comments extraction
Video:  5
Opening Video:  5
Switching driver to Video:  5
Switching page for Video:  5
Getting number of comment

# Scraping Articles using - News API
  Steps:
     (Step1)  : Generate API Key for NewsApiClient
     (Step2)  : Call newsapi.get_everything with query q as 'business analytics' and sort_by='publishedAt' to sort 
               by recent ones
     (Step 3) : In response returned, check totalResults returned to find number of matching articles
     (Step 4) : newsAPI returns 20 articles per page. So, in results returned, loop through        totalResults/defaultPageSize times. Note, defaultPageSize = 20
     (Step 5) : Each time in for loop, read fields - source id, source name, author, title, description and content
     (Step 6) : Write above article fields to articleRow
     (Step 7) : Append each of the articles to articleList
     (Step 8) : newsapi limits number of matching article responses to 1000 (till Page=50)
     (Step 9) : Convert articleList to dataframe object
     (Step 10): Write articleDF object to output_api file

URL: https://newsapi.org/v2/everything?q=business%20analytics&apiKey=3a8a868b31cd4975973a0cab97e98a91&language=en

APIKEY: 3a8a868b31cd4975973a0cab97e98a91


In [ ]:
import pandas as pd
from newsapi import NewsApiClient
try:
    maxfreeArticlesPageCountLimit = 50
    defaultPageSize = 20
    newsapi = NewsApiClient(api_key='3a8a868b31cd4975973a0cab97e98a91')
    newsapiResponse = newsapi.get_everything(q='business analytics',
                                      language='en',
                                      sort_by='publishedAt')
    totalResults = newsapiResponse['totalResults']
    if(totalResults > 0):
        numOfArticlePages = totalResults/defaultPageSize
        pageNumber = 1
        articlesList = []
        while (pageNumber <= numOfArticlePages) and (pageNumber <= maxfreeArticlesPageCountLimit):
            newsapiResponse = newsapi.get_everything(q='business analytics',
                                      language='en',
                                      sort_by='publishedAt',
                                      page=pageNumber)
            newapiResponseStatus = newsapiResponse['status']
            if(newapiResponseStatus == 'ok'):
                numOfArticlesInCurrentPage = len(newsapiResponse['articles'])
                pageLevelArticleCounter = 0
                while pageLevelArticleCounter < numOfArticlesInCurrentPage:
                    articleRow = []
                    articleRow.append(newsapiResponse['articles'][pageLevelArticleCounter]['source']['id'])
                    articleRow.append(newsapiResponse['articles'][pageLevelArticleCounter]['source']['name'])
                    articleRow.append(newsapiResponse['articles'][pageLevelArticleCounter]['author'])
                    articleRow.append(newsapiResponse['articles'][pageLevelArticleCounter]['title'])
                    articleRow.append(newsapiResponse['articles'][pageLevelArticleCounter]['description'])
                    articleRow.append(newsapiResponse['articles'][pageLevelArticleCounter]['content'])
                    articlesList.append(articleRow)
                    pageLevelArticleCounter += 1
                pageNumber += 1    
        if(len(articlesList)>0):
            columns_list = ["Source ID","Source Name","Auhtor","Title","Description","Contents"]
            articleDF = pd.DataFrame(articlesList, columns=columns_list)   
            articleDF.to_csv("output_api.csv", index=False, columns=columns_list)
            print(str(totalResults)+" articles match found.")
            print("Successfully Inserted: "+str(len(articlesList))+" articles to  output_api.csv file based on maxfree limit permitted by newsAPI")
        else:
            print('No articles found')
    else:
        print('totalResults returned: '+str(totalResults)+' is not > 0')
except Exception as e:
    print('Exception raised: '+str(e))
